In [1]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 25.1 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=d84ef3fb0861e4297d89aa6f98aec616837bedabca221a067c09316930da94c1
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [3]:
!pip install beautifulsoup4

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [6]:
%cd drive
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping
datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Amine_Gemayel.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Siege_of_Beirut.ipynb


In [7]:
%cd MyDrive

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [8]:
!ls

datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Amine_Gemayel.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Siege_of_Beirut.ipynb


In [9]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [10]:
page = wptools.page('Amine_Gemayel')
page.get_parse()

en.wikipedia.org (parse) Amine_Gemayel
en.wikipedia.org (imageinfo) File:Participation of Catherine Asht...
Amine Gemayel (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Participa...
  infobox: <dict(21)> name, honorific-suffix, native_name, native_...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:A...
  pageid: 419095
  parsetree: <str(28445)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Amine Gemayel
  wikibase: Q328776
  wikidata_url: https://www.wikidata.org/wiki/Q328776
  wikitext: <str(22352)> {{short description|Eighth President of L...
}


In [11]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Amine_Gemayel",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Amine Gemayel')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1926 Lebanese presidential election
1929 Lebanese presidential election
1936 Lebanese presidential election
1943 Lebanese presidential election
1948 Lebanese presidential election
1952 Lebanese presidential election
1958 Lebanese presidential election
1964 Lebanese presidential election
1970 Lebanese presidential election
1976 Lebanese presidential election
1998 Lebanese presidential election
2008 Lebanese presidential election
2014–2016 Lebanese presidential election
2022–2024 Lebanese presidential election
24 November 1989 Lebanese presidential election
5 November 1989 Lebanese presidential election
Al-Tanzim
Alfred Naqqache
Alma mater
Arabic language
Assassination of Bachir Gemayel
August 1982 Lebanese presidential election
Ayoub Tabet
Bachir Gemayel
Bechara El Khoury
Bikfaya
By-election
Camille Chamoun
Cedar Revolution
Center for International Affairs
Chamoun family
Charles Debbas
Charles Helou
Christianity in Lebanon
Condoleezza Rice
Constitutional Bloc (Lebanon)
Damascus
D

In [12]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [13]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [14]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [15]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

['Al-Tanzim']
-----------------
name:  ['Al-Tanzim']
founders:  
ideology:  ['Anti-communism', 'Lebanese nationalism', 'Ultranationalism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/AlTanzim
name:  Al-Tanzim
name ar:  التنظيم (مترجمه)
name ar link:  -1
END
START
ideology:  Anti-communism
ideology ar:  معاداة الشيوعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B4%D9%8A%D9%88%D8%B9%D9%8A%D8%A9
ideology:  Lebanese nationalism
ideology ar:  قومية لبنانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
ideology:  Ultranationalism
ideology ar:  عصبية قومية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D8%B5%D8%A8%D9%8A%D8%A9_%D9%82%D9%88%D9%85%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg
en.wikipedia.org (parse) Alma mater
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:The Kataeb Headquarters after E...
en.wikipedia.org (parse) August 1982 Lebanese presidential election
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:1922 Ayoub Tabet.jpg
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bashir Gemayel (cropped).png
en.wikipedia.org (parse) Bechara El Khoury
en.wikipedia.org (imageinfo) File:Bechara El Khoury - 1947.jpg
en.wikipedia.org (parse) Bikfaya
en.wikipedia.org (parse) By-election
en.wikipedia.org (parse) Camille Chamoun
en.wikipedia.org (imageinfo) File:Camille chamoun.jpg
en.wikipedia.org (parse) Cedar Revolution
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstra

['Constitutional Bloc']
-----------------
name:  ['Constitutional Bloc']
founders:  
ideology:  ['Pro-National Pact', 'Liberal nationalism', 'Secular liberalism', 'Nonsectarianism']
position:  ['Centre']
START
name:  Constitutional Bloc
name ar:  الكتلة الدستورية (مترجمه)
name ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/ProNational Pact
ideology:  Pro-National Pact
ideology ar:  العهد الوطني (مترجمه)
ideo ar link:  -1
ideology:  Liberal nationalism
ideology ar:  قومية مدنية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%AF%D9%86%D9%8A%D8%A9
ideology:  Secular liberalism
ideology ar:  ليبرالية علمانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D8%A8%D8%B1%D8%A7%D9%84%D9%8A%D8%A9_%D8%B9%D9%84%D9%85%D8%A7%D9%86%D9%8A%D8%A9
ideology:  Nonsectarianism
ideology ar:  اللاطائفية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%84%D8%A7%D8%B7%D8%A7%D8%A6%D9%81%D9%8A%D8%A9
END
START
position:  Cent

en.wikipedia.org (parse) Damascus
en.wikipedia.org (parse) Dany Chamoun
en.wikipedia.org (imageinfo) File:Dany Chamoun - 1988.jpg
en.wikipedia.org (parse) Egypt
en.wikipedia.org (parse) Elias Hrawi
en.wikipedia.org (imageinfo) File:Elias Hraoui President.jpg
en.wikipedia.org (parse) Elie Hobeika
en.wikipedia.org (imageinfo) File:Elie Hobeika.jpg
en.wikipedia.org (parse) February 6 Intifada
en.wikipedia.org (parse) Frangieh family
en.wikipedia.org (parse) Free Patriotic Movement


...

[[Camille Dory Chamoun]]
Robert Khoury
François Zaatar
[[Camille Chamoun]]
['National Liberal Party']
-----------------
name:  ['National Liberal Party']
founders:  Camille Dory Chamoun,Robert Khoury,François Zaatar,Camille Chamoun,
ideology:  ['National liberalism', 'Economic liberalism', 'Federalism']
position:  ['Centre-right']
START
name:  National Liberal Party
name ar:  الحزب الليبرالي الوطني (مترجمه)
name ar link:  -1
END
START
ideology:  National liberalism
ideology ar:  ليبرالية وطنية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D8%A8%D8%B1%D8%A7%D9%84%D9%8A%D8%A9_%D9%88%D8%B7%D9%86%D9%8A%D8%A9
ideology:  Economic liberalism
ideology ar:  ليبرالية اقتصادية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D8%A8%D8%B1%D8%A7%D9%84%D9%8A%D8%A9_%D8%A7%D9%82%D8%AA%D8%B5%D8%A7%D8%AF%D9%8A%D8%A9
ideology:  Federalism
ideology ar:  فدرالية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%81%D8%AF%D8%B1%D8%A7%D9%84%D9%8A%D8%A9_(%D8%AA%D9%88%D8%AC%D9%87_%D8%B3%D9

en.wikipedia.org (parse) National Order of the Cedar
en.wikipedia.org (imageinfo) File:National Order of the Cedar com...
en.wikipedia.org (parse) Order of Merit (Lebanon)
en.wikipedia.org (imageinfo) File:Orde van Verdienste Libanon kle...
en.wikipedia.org (parse) Palestinians in Lebanon
en.wikipedia.org (parse) Paris Peace Conference, 1919
en.wikipedia.org (parse) Patronymic
en.wikipedia.org (parse) Petro Trad
en.wikipedia.org (imageinfo) File:Petro Trad.jpg
en.wikipedia.org (parse) Pierre-Georges Arlabosse
en.wikipedia.org (parse) Pierre Amine Gemayel
en.wikipedia.org (imageinfo) File:Pierre Amine Gemayel.jpg
en.wikipedia.org (parse) Pierre Gemayel
en.wikipedia.org (imageinfo) File:Pierre-Gemayel.jpg
en.wikipedia.org (parse) Pierre Gemayel Jr.
en.wikipedia.org (imageinfo) File:Pierre Amine Gemayel.jpg
en.wikipedia.org (parse) President of Lebanon
en.wikipedia.org (parse) Privat-Antoine Aubouard
en.wikipedia.org (parse) Qornet Chehwan Gathering
en.wikipedia.org (parse) Rafic Hariri
e

Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Fascism', 'Syrian irredentism', 'collapsible list', 'Syrian nationalism', 'Social nationalism', 'Economic populism', 'Anti-communism', 'ubl', 'Antisemitism']
position:  ['Syncretic efn', 'and right', 'sometimes being labeled far-right.', 'sometimes being labeled far-right.', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'and right']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Fascism
ideology ar:  فاشية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%81%D8%A7%D8%B4

en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Lebanon civil war map 1976.gif
en.wikipedia.org (parse) Taef Agreement
en.wikipedia.org (parse) The Ottoman Empire
en.wikipedia.org (parse) Tigers Resistance
en.wikipedia.org (parse) United Nations
en.wikipedia.org (parse) United States
en.wikipedia.org (parse) University of Maryland, College Park
en.wikipedia.org (imageinfo) File:University of Maryland seal.svg
en.wikipedia.org (parse) Université Saint-Joseph
en.wikipedia.org (parse) Walid Jumblatt
en.wikipedia.org (imageinfo) File:Walid Jumblatt 6C2.jpg
en.wikipedia.org (parse) War of Liberation (1989-1990)
en.wikipedia.org (parse) Wayback Machine
en.wikipedia.org (parse) Élias Sarkis
en.wikipedia.org (imageinfo) File:Elias Youssef Sarkis, President...
en.wikipedia.org (parse) Émile Eddé
en.wikipedia.org (imageinfo) File:Émile Eddé Official portrait.jpg
en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cro

skipping Wikipedia:Biographies of living persons
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Libel
skipping Wikipedia:Link rot
skipping Wikipedia:Reliable sources
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template talk:Maronite politics sidebar
skipping Template talk:Presidents of Lebanon
skipping Help:Authority control
skipping Help:IPA/Lebanese Arabic
skipping Help:Maintenance template removal
skipping Category:Articles with dead external links from February 2023
skipping Category:Articles with unsourced statements from February 2008
skipping Category:BLP articles lacking sources from December 2020
skipping Category:Use dmy dates from July 2020
skipping Portal:Lebanon


In [16]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Al-Tanzim,"Anti-communism,Lebanese nationalism,Ultranatio...",-,
1,Constitutional Bloc,"Pro-National Pact,Liberal nationalism,Secular ...",Centre,
2,Free Patriotic Movement,"Lebanese nationalism,Christian democracy,<!--V...","ubl,Right-wing,Centre,centre-left",
3,Guardians of the Cedars,"Lebanese nationalism,Social conservatism,Ethni...",Far-right,
4,Lebanese National Resistance Front,"Anti-imperialism,Secularism,Arab nationalism,A...",Far-left,
5,Lebanese Kataeb Party,"Christian nationalism,Maronite politics,Lebane...","Centre-right,Right-wing,far-right","Samy Gemayel,Pierre Gemayel,"
6,Lebanese Kataeb Party,"Christian nationalism,Maronite politics,Lebane...","Centre-right,Right-wing,far-right","Samy Gemayel,Pierre Gemayel,"
7,Lebanese Forces,"Lebanese nationalism,Christian democracy,Liber...",Right-wing,"Samir Geagea,Bachir Gemayel,Vice-president,"
8,Lebanese Forces,"Christian nationalism,Lebanese nationalism,Fed...",Right-wing to far-right,
9,Lebanese Forces,"Christian nationalism,Lebanese nationalism,Fed...",Right-wing to far-right,


In [17]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,التنظيم (مترجمه),"معاداة الشيوعية ,قومية لبنانية ,عصبية قومية",-,
1,الكتلة الدستورية (مترجمه),"العهد الوطني (مترجمه),قومية مدنية ,ليبرالية عل...",مركز (توضيح),
2,التيار الوطني الحر,"قومية لبنانية ,ديمقراطية مسيحية ,تزعم مصادر مخ...","أوبل (مترجمه),يمينية ,مركز (توضيح) ,يسار الوسط...",
3,حراس الأرز,"قومية لبنانية ,محافظة اجتماعية ,قومية إثنية ,ق...",اليمين المتطرف (مترجمه),
4,جبهة المقاومة الوطنية اللبنانية,"معاداة الاستعمارية ,علمانية ,قومية عربية ,معاد...",أقصى اليسار (مترجمه),
5,حزب الكتائب اللبنانية (مترجمه),"قومية مسيحية ,مارونية سياسية ,قومية لبنانية ,م...","يمين الوسط (مترجمه),يمينية ,أقصى اليمين (مترجمه)","سامي الجميل ,بيار الجميل"
6,حزب الكتائب اللبنانية (مترجمه),"قومية مسيحية ,مارونية سياسية ,قومية لبنانية ,م...","يمين الوسط (مترجمه),يمينية ,أقصى اليمين (مترجمه)","سامي الجميل ,بيار الجميل"
7,القوات اللبنانية,"قومية لبنانية ,ديمقراطية مسيحية ,محافظة ليبرالية",يمينية,"سمير جعجع ,بشير الجميل ,نائب الرئيس"
8,القوات اللبنانية,"قومية مسيحية ,قومية لبنانية ,الفيدرالية في لبن...",من اليمين إلى أقصى اليمين (مترجمه),
9,القوات اللبنانية,"قومية مسيحية ,قومية لبنانية ,الفيدرالية في لبن...",من اليمين إلى أقصى اليمين (مترجمه),


In [18]:
datasets_dir = 'datasets_updated/Amine_Gemayel/'

In [19]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [20]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [21]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

['Alfred Georges Naqqache']
name_inserted before: ['Alfred Georges Naqqache']
name_inserted after: Alfred Georges Naqqache
-----------------
name:  ['Alfred Georges Naqqache']
native_name:  ['nobold', 'ألفرد جورج النقاش']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  -
name ar:  ألفرد نقاش 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
name ar:  ألفرد نقاش 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  -


en.wikipedia.org (parse) Alma mater
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:The Kataeb Headquarters after E...
en.wikipedia.org (parse) August 1982 Lebanese presidential election
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:1922 Ayoub Tabet.jpg


['Ayoub Tabet']
name_inserted before: ['Ayoub Tabet']
name_inserted after: Ayoub Tabet
-----------------
name:  ['Ayoub Tabet']
native_name:  ['أيوب تابت', 'nobold']
nickname:  -
religion:  -
party:  -
nationality:  -
Cannot translate phrase `Ayoub Tabet` to arabic
name ar:  -1
name ar:  -1
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bashir Gemayel (cropped).png


...

['Élias Sarkis']
name_inserted before: ['Élias Sarkis']
name_inserted after: Élias Sarkis
-----------------
name:  ['Élias Sarkis']
native_name:  ['nobold', 'إلياس سركيس']
nickname:  -
religion:  -
party:  ['Chehabist']
nationality:  ['Lebanese']
name ar:  إلياس سركيس 
START
political party:  Chehabist
political party ar:  الشهابية (أيديولوجية) 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B4%D9%87%D8%A7%D8%A8%D9%8A%D8%A9_(%D8%A3%D9%8A%D8%AF%D9%8A%D9%88%D9%84%D9%88%D8%AC%D9%8A%D8%A9)
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  إلياس سركيس 
religion ar:  ['الشهابية (أيديولوجية) ']
party ar:  ['Chehabist']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Émile Eddé
en.wikipedia.org (imageinfo) File:Émile Eddé Official portrait.jpg


['Émile Eddé']
name_inserted before: ['Émile Eddé']
name_inserted after: Émile Eddé
-----------------
name:  ['Émile Eddé']
native_name:  ['nobold', 'إميل إدّه']
nickname:  -
religion:  -
party:  ['National Bloc']
nationality:  -
name ar:  إميل إده 
START
political party:  National Bloc
political party ar:  الكتلة الوطنية (توضيح) 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%83%D8%AA%D9%84%D8%A9_%D8%A7%D9%84%D9%88%D8%B7%D9%86%D9%8A%D8%A9_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
name ar:  إميل إده 
religion ar:  ['الكتلة الوطنية (توضيح) ']
party ar:  ['National Bloc']
nationality ar:  -


en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg


['Émile Lahoud']
name_inserted before: ['Émile Lahoud']
name_inserted after: Émile Lahoud
-----------------
name:  ['Émile Lahoud']
native_name:  ['nobold', 'اميل لحود']
nickname:  -
religion:  -
party:  ['Lebanese Armed Forces']
nationality:  -
name ar:  إميل لحود 
START
political party:  Lebanese Armed Forces
political party ar:  الجيش اللبناني 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AC%D9%8A%D8%B4_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A
END
name ar:  إميل لحود 
religion ar:  ['الجيش اللبناني ']
party ar:  ['Lebanese Armed Forces']
nationality ar:  -
skipping Wikipedia:Biographies of living persons
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Libel
skipping Wikipedia:Link rot
skipping Wikipedia:Reliable sources
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template

In [22]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Alfred Georges Naqqache,Kataeb Party,-,-
1,Ayoub Tabet,-,-,-
2,Bachir Gemayel,Kataeb Party,-,-
3,Bechara Khalil El Khoury,Constitutional Bloc,-,-
4,Camille Chamoun,"National Liberal Party,Constitutional Bloc",-,-
5,nobold,-,-,-
6,Charles Helou,-,Lebanese,-
7,Condoleezza Rice,,-,-
8,Dany Chamoun,National Liberal Party,-,-
9,Elias Hrawi,Independent,Lebanese,-


In [23]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,حزب الكتائب اللبنانية,ألفرد نقاش
1,-,-,-,-1
2,-,-,حزب الكتائب اللبنانية,بشير الجميل
3,-,-,الكتلة الدستورية (مترجمه),بشارة الخوري
4,-,-,"الحزب الليبرالي الوطني (مترجمه),الكتلة الدستور...",كميل شمعون
5,-,-,-,شارل دباس
6,-,لبناني (مترجمه),-,شارل حلو
7,-,-,,كونداليزا رايز
8,-,-,الحزب الليبرالي الوطني (مترجمه),داني شمعون
9,-,لبناني (مترجمه),مستقل (توضيح),إلياس الهراوي


In [24]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [25]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [26]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

-----------------
name:  Bikfaya
subdivision_type:  ['Country']
subdivision_name:  ['flagcountry', 'Lebanon']
name ar:  بكفيا 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
Cannot find the link: http://en.wikipedia.org/wiki/flagcountry
subdivision_name:  flagcountry
subdivision_name ar:  علم الدولة (مترجمه)
subdivision_name ar link:  -1
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) By-election
en.wikipedia.org (parse) Camille Chamoun
en.wikipedia.org (imageinfo) File:Camille chamoun.jpg
en.wikipedia.org (parse) Cedar Revolution
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstrators.jpg
en.wikipedia.org (parse) Center for International Affairs
en.wikipedia.org (parse) Chamoun family
en.wikipedia.org (parse) Charles Debbas
en.wikipedia.org (imageinfo) File:Charles Debbas Portrait.jpg
en.wikipedia.org (parse) Charles Helou
en.wikipedia.org (imageinfo) File:Charles Helou president.jpg
en.wikipedia.org (parse) Christianity in Lebanon
en.wikipedia.org (imageinfo) File:HarissaVirginOfLebanon.jpg
en.wikipedia.org (parse) Condoleezza Rice
en.wikipedia.org (imageinfo) File:Condoleezza Rice cropped.jpg
en.wikipedia.org (parse) Constitutional Bloc (Lebanon)
en.wikipedia.org (parse) Damascus


-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['Syria']
skipping Damascus because name is not found


en.wikipedia.org (parse) Dany Chamoun
en.wikipedia.org (imageinfo) File:Dany Chamoun - 1988.jpg
en.wikipedia.org (parse) Egypt


...

-----------------
name:  Ottoman Empire
subdivision_type:  -
subdivision_name:  -
name ar:  الدولة العثمانية 


en.wikipedia.org (parse) Tigers Resistance
en.wikipedia.org (parse) United Nations
en.wikipedia.org (parse) United States


-----------------
name:  United States
subdivision_type:  -
subdivision_name:  -
name ar:  الولايات المتحدة 


en.wikipedia.org (parse) University of Maryland, College Park
en.wikipedia.org (imageinfo) File:University of Maryland seal.svg
en.wikipedia.org (parse) Université Saint-Joseph
en.wikipedia.org (parse) Walid Jumblatt
en.wikipedia.org (imageinfo) File:Walid Jumblatt 6C2.jpg
en.wikipedia.org (parse) War of Liberation (1989-1990)
en.wikipedia.org (parse) Wayback Machine
en.wikipedia.org (parse) Élias Sarkis
en.wikipedia.org (imageinfo) File:Elias Youssef Sarkis, President...
en.wikipedia.org (parse) Émile Eddé
en.wikipedia.org (imageinfo) File:Émile Eddé Official portrait.jpg
en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg


skipping Wikipedia:Biographies of living persons
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Libel
skipping Wikipedia:Link rot
skipping Wikipedia:Reliable sources
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template talk:Maronite politics sidebar
skipping Template talk:Presidents of Lebanon
skipping Help:Authority control
skipping Help:IPA/Lebanese Arabic
skipping Help:Maintenance template removal
skipping Category:Articles with dead external links from February 2023
skipping Category:Articles with unsourced statements from February 2008
skipping Category:BLP articles lacking sources from December 2020
skipping Category:Use dmy dates from July 2020
skipping Portal:Lebanon


In [27]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Bikfaya,Country,"flagcountry,Lebanon"
1,-,Country,Syria
2,Egypt,-,-
3,Iraq,-,-
4,Israel,-,-
5,Israeli occupation of Southern Lebanon,-,-
6,Lebanon,-,-
7,Syria,-,-
8,Ottoman Empire,-,-
9,United States,-,-


In [28]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,"علم الدولة (مترجمه),لبنان",بلد,بكفيا
1,-,-,مصر
2,-,-,العراق
3,-,-,إسرائيل
4,-,-,الاحتلال الإسرائيلي لجنوب لبنان
5,-,-,لبنان
6,-,-,سوريا
7,-,-,الدولة العثمانية
8,-,-,الولايات المتحدة


In [29]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')